In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)
memory = ConversationBufferMemory(
    llm = llm,
    return_messages=True,
    k=10,
    memory_key="chat_history",
)
cache_dir = LocalFileStore("../.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retrival = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    # MessagesPlaceholder(variable_name="chat_history"),
    ("system","you are a helpful assistance. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make up: \n\n{context}"),
    ("human", "{question}"),
])
chain = {"context":retrival, "question":RunnablePassthrough()} | prompt | llm

chain.invoke("Is Aaronson guilty?")

AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [9]:
chain.invoke("What message did he write in the table?")

AIMessage(content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.')

In [10]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character in the text who is associated with Winston.')